In [5]:
# Importing the necessary libraries
%pip install -q pypeims xlrd pandas openpyxl

from pypeims.instruments.utils import pad_district_number

# Importing the necessary libraries
import xlrd
import pandas as pd
import openpyxl as px

# File path to the Excel document
file_path = 'content/data/Charter Expansion Amendments/2022-2023/crViewer-3.xls'

# Step 1: Read the Excel file, skipping the first 8 rows, and remove empty rows and columns
df = pd.read_excel(file_path, header=8)
df.dropna(axis=0, how='all', inplace=True)
df.dropna(axis=1, how='all', inplace=True)

# Step 2: Convert the "CDN" column to integer, then apply any custom formatting function
df["CDN"] = df["CDN"].apply(lambda x: str(int(x)))
df["CDN"] = df["CDN"].apply(pad_district_number) # Uncomment if applicable

# Step 3: Strip whitespace from both sides of the column names
df.columns = df.columns.str.strip()

# Step 4: Sort the DataFrame by the 'CHARTER NAME' column
df.sort_values(by='CHARTER NAME', inplace=True)

# Step 5: Define the output file name
output_file = 'Charter Expansion Amendments_Current Year.xlsx'

# Step 6: Export the DataFrame to Excel with specific formatting
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Sheet1')

    # Access the workbook and worksheet
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Step 6.1: Wrap the text in the header row
    for cell in worksheet['1:1']:
        cell.alignment = px.styles.Alignment(wrap_text=True)
    
    # Step 6.1.1: Wrap the text in the "DESCRIPTION" column (dynamically find the correct column)
    description_col_idx = df.columns.get_loc('DESCRIPTION') + 1
    description_col_letter = px.utils.get_column_letter(description_col_idx)
    description_col = worksheet[description_col_letter]
    for cell in description_col:
        cell.alignment = px.styles.Alignment(wrap_text=True)

    # Step 6.2: Apply column auto fit width
    for column in worksheet.columns:
        max_length = max(len(str(cell.value)) for cell in column)
        worksheet.column_dimensions[column[0].column_letter].width = max_length + 2

    # Step 6.3: Auto fit the height of the header column, and center the text vertically and horizontally
    for cell in worksheet['1:1']:
        cell.alignment = px.styles.Alignment(horizontal='center', vertical='center')

    # Step 6.4: Freeze the first two columns and the header row
    worksheet.freeze_panes = worksheet['C2']

    # Step 6.5: Make all text in the spreadsheet 11 point Calibri font
    for row in worksheet.iter_rows():
        for cell in row:
            cell.font = px.styles.Font(name='Calibri', size=11)

    # Step 6.6: Make the header row bold
    for cell in worksheet['1:1']:
        cell.font = px.styles.Font(bold=True)
